<a href="https://colab.research.google.com/github/nitsundon/Load-Forecast/blob/main/ReadDSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install tesseract-ocr
!pip install requests pillow pytesseract

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
import drive.MyDrive.Libraries.SV as sv

In [10]:
sv.getDemand()

'2025.04.04 22:35 --> 26147'

In [9]:
# prompt: create a sqllite database and store google drive

import sqlite3
import os

# Define the path to your database file in Google Drive
db_path = '/content/drive/MyDrive/Libraries/my_database.db'

# Create the database file if it doesn't exist
if not os.path.exists(db_path):
  conn = sqlite3.connect(db_path)
  cursor = conn.cursor()

  # Create a table (example)
  cursor.execute('''
      CREATE TABLE IF NOT EXISTS demand_data (
          id INTEGER PRIMARY KEY AUTOINCREMENT,
          time TEXT,
          demand TEXT
      )
  ''')
  conn.commit()
  conn.close()


